In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import missingno as msno
from survey_data import SurveyData
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scikit_posthocs as sp
import plotly.express as px



from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from scipy import stats
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# from textblob import TextBlob
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud



# 1. Load and clean data

In [ ]:
#  build folder for each part (automated)

folders = ['Graphs/Qs', 'Graphs/Rating', 'Graphs/Missing data', 'Graphs/Feedbacks', 'Graphs/Satisfaction', 'Graphs/Feedbacks']

# Create the directory
for directory in folders:
    os.makedirs(directory, exist_ok=True)

In [ ]:
dep_df_list = []  # each element of the list contains the data of a department
source_list = ["Business School", "CM", "CQT", "ETP", "ISS", "IVO", "LKYSPP", "Medical School", "ODPRT", "SCALE", "UCI"]
for source in source_list:
    location = f'Raw Results/NUS {source}.xlsx'
    df = SurveyData(source, location)
    df_raw = df.load_data()
    df_dropped = df.data_drop(df_raw)
    # df_dropped = df_dropped.drop('Progress', axis=1)
    df_coded = df.coding(df_dropped)
    # print(df_coded)
    df_no_rank = df.del_rank(df_coded)
    # df_filled = df_no_rank.fillna(0)
    # df_filled.head()
    dep_df_list.append(df_coded)

df_overall = pd.concat(dep_df_list, axis=0, ignore_index = True)
print(df_overall.head())
print(f'overall df info:\n{df_overall.info()}')
print(f'count:\n{df_overall.count()}')
df_overall_Q = df_overall.iloc[:,3:-1]
# print("describe overall data:")
# print(df_overall.describe())
# merge_data.visualize_Qs(df_overall)

## 1.1 Missing data

In [ ]:
def draw_missing_data_perc(df, data_source):
    plt.figure(figsize=(12,6))
    sns.barplot(df.isna().sum()/len(df)*100)
    plt.axhline(y=5, color='red', linestyle='--')
    plt.xticks(rotation=45)
    plt.title(f'Percentage of {data_source} missing data')
    plt.ylabel('percentage of missing data (%)')
    plt.savefig(f'Graphs/Missing data/Percentage of {data_source} missing data.png')
    plt.show()

### Overall Missing data for each Q
If missing data is more than 5%, instead of deleting them, other options should be taken.

In [ ]:
# overall num
print(f'overall missing data:\n {df_overall.isna().sum()}\n')

# visualiza overall missing data for each Q
draw_missing_data_perc(df_overall, 'overall')
# tips: savefig before show


# deal with the missing values
# percentage of missing rows
max_missing_Q = df_overall_Q.isna().any(axis=1).sum()  # if there is at least one True (i.e., at least one NaN) across all columns
percentage_missing = max_missing_Q/len(df_overall) * 100
print(f'max percentage of missing data (rows): {percentage_missing:.2f}%, overall num is {len(df_overall)}, the max missing num of Question is {max_missing_Q}')
df_overall_cleaned = df_overall_Q[~df_overall_Q.isna().any(axis=1).values]
df_overall_cleaned = df_overall_cleaned.reset_index(drop=True)
overall_num = len(df_overall_cleaned)
print(f'length of cleaned overall data: {overall_num}')
print(f'head of overall cleaned data:\n{df_overall_cleaned.head()}')

dep_df_cleaned_list = []
for i in range(len(source_list)):
    dep_df_Q = dep_df_list[i].iloc[:,3:-1]
    dep_df_cleaned = dep_df_Q[~dep_df_Q.isna().any(axis=1).values]
    dep_df_cleaned = dep_df_cleaned.reset_index(drop=True)
    dep_df_cleaned_list.append(dep_df_cleaned)
print(f'cleaned data for first department: \n{dep_df_cleaned_list[0]}')

### Missing data for each department

In [ ]:
# each department num
for i,dep in enumerate(source_list):
    department = source_list[i]
    dep_df = dep_df_list[i]
    # print(f'{department} missing data:\n{dep_df.isna().sum()}\n')
    draw_missing_data_perc(dep_df, dep + ' department')


# 2. Descriptive statistics 

## 2.1 Count, percentage (Overall Satisfaction,SP,SN)

In [ ]:
def visualize_distri_each_Q(y, y_label,path,average=False):
    plt.clf()
    x = ['Q'+str(i) for i in range(1,16)]
    colors = ['blue'] * len(x)
    min_idx = y.index(min(y))
    max_idx = y.index(max(y))
    # Change the color of the bar at index 2 (bar for 'C') to red
    colors[min_idx] = 'red'
    colors[max_idx] = 'green'
    plt.bar(x,y, color = colors, alpha=0.6)
    if average == True:
        m = float(np.mean(y))
        plt.axhline(m, color='grey', linestyle='--', linewidth=2, label='Average')
        plt.text(x=x[0],y=m,s=f'{round(m,2)}',color='grey')
        plt.legend()
    for i in range(len(x)):
        plt.text(x=x[i], 
                y=y[i], 
                s=f"{y[i]}", 
                ha='center', 
                va='bottom')
    plt.xticks(rotation=45, ha='right') # Rotate x labels for better readability
    y_upper = max(y)+0.01
    y_lower = max(min(y)-0.01,0)
    # print(f'y range: {y_lower} to {y_upper}')
    plt.ylim(y_lower,y_upper)
    plt.xlabel('Questions')
    plt.ylabel(y_label)
    plt.title(f'{y_label} in each Question')
    plt.show()
    plt.savefig(path)


In [ ]:
Q_cols = ['Q'+str(i) for i in range(1,16)]
combined_cols = Q_cols + ['Department']
combined_dfs = pd.DataFrame(columns=combined_cols)  # combine all data into one df


for idx,dep in enumerate(source_list):
    df_dep = dep_df_cleaned_list[idx]
    df_dep['Department'] = dep
    # Concatenate along rows (axis=0)
    combined_dfs = pd.concat([combined_dfs, df_dep], axis=0, ignore_index=True)
print(f'head of combined_dfs:\n{combined_dfs.head()}')

# answer of each Q (performance, how satisfied about it)
Q_Score_count = [combined_dfs[x].value_counts().reset_index() for x in Q_cols]
for c in Q_Score_count:
    c['percentage'] = round(c['count']/overall_num,2)
    c.columns = ['score','count','percentage']
print(Q_Score_count)

# for each question (satisfaction)
Q_performance = [round(df.sort_values(by='score', ascending=False).loc[0,'percentage'] + df.sort_values(by='score', ascending=False).loc[1, 'percentage'],2) for df in Q_Score_count]
y_label = 'Overall Satisfaction (Strongly positive + positive)'
visualize_distri_each_Q(Q_performance, y_label, f'Graphs/Satisfaction/{y_label}.png',average=True)

Q_SP = [df.sort_values(by=['score'], ascending=False).reset_index().loc[0,'percentage'] for df in Q_Score_count]
print('Q_SP:', Q_SP) 
y_label = 'Q_SP(strongly positive)'
visualize_distri_each_Q(Q_SP, y_label, f'Graphs/Satisfaction/{y_label}.png')

Q_SN = [df.sort_values(by='score', ascending=True).reset_index().loc[0,'percentage'] for df in Q_Score_count]
print('Q_SN:', Q_SN) 
y_label = 'Q_SN(strongly negative)'
visualize_distri_each_Q(Q_SN, y_label, f'Graphs/Satisfaction/{y_label}.png')

## 2.2 Mean

In [ ]:
# each Q
Q_mean_df = combined_dfs[Q_cols].mean().reset_index()
Q_mean_df.columns = ['Q', 'mean']
Q_mean_df['mean']= Q_mean_df['mean'].astype(float)
Q_mean_df['mean'] = round(Q_mean_df['mean'],2)

# Q_mean_df

visualize_distri_each_Q(Q_mean_df['mean'].tolist(), 'Mean value', 'Graphs/Qs/Mean for Each Question.png',average=True)

## conclusion: Q6 is the lowest, while Q2 hightest




## 2.3 Variation
Not very useful in this case: all answers are categorical variables with 5 possible values, so it doesn't make much sense to see how they vary. But it can still show the distribution to some degree.

In [ ]:
Q_std_df = combined_dfs[Q_cols].std().reset_index()
Q_std_df.columns = ['Q', 'std']
Q_std_df['std']= Q_std_df['std'].astype(float)
Q_std_df['std'] = round(Q_std_df['std'],2)

# Q_std_df

visualize_distri_each_Q(Q_std_df['std'].tolist(), 'Standard deviation', 'Graphs/Qs/Std for Each Question.png')


## conclusion: pretty close, not a good indicator

## 2.4 CV (Coefficient of variation)
As each question has same categorical variable answer, variance related metrics cannot tell us much about the distribtion (central tendency)

In [ ]:
# CV for each question (scale): also help to determine the weights
combined_df = pd.merge(Q_std_df, Q_mean_df, on='Q')
Q_CV_df = combined_df.copy()
Q_CV_df['CV'] = round((Q_CV_df['std'] / Q_CV_df['mean']) ,2)
# Q_CV_df = Q_CV_df.sort_values(by=['CV','mean'], ascending=False).reset_index(drop=True)

print(f'Q_CV_df_sorted:\n{Q_CV_df}')
# CV_df.to_excel('Qs.xlsx', index=False)

visualize_distri_each_Q(Q_CV_df['CV'].tolist(), 'CV', 'Graphs/Qs/CV for Each Question.png')




## 2.5 Percentage of neutral


In [ ]:
# percentage of neutral for each Q
def neutral_percentage(combined_dfs,Q_cols):
    neu_perc_list = []
    for q in Q_cols:
        neutral_num = combined_dfs[[q]].value_counts()[0]
        overall_num = combined_dfs[q].count()
        neutral_perc = round(neutral_num/overall_num,2)
        neu_perc_list.append(neutral_perc)
    print(f'percentage of neutral answers over the overall answers:{neu_perc_list}')
    visualize_distri_each_Q(neu_perc_list, 'Neutral percentage','Graphs/Qs/Neutral percentage of each Q',average=True)
    high_perc = [i for i, perc in enumerate(neu_perc_list) if perc > 0.2]
    high_perc_Qs = [Q_cols[i] for i in high_perc]
    # for Qs that have high neutral percentage, see the department
    neu_dep_df = pd.DataFrame(columns=['Department'] + high_perc_Qs)
    
    for h in high_perc_Qs:
        df = combined_dfs[[h,'Department']]
        # percentage of neutral in the department
        count_df = df.groupby('Department')[h].apply(lambda x: (x == 0).sum()).reset_index(name='ZeroCount')
        popu_df = df.groupby('Department').size().reset_index(name='Population')
        count_df = pd.merge(count_df, popu_df, on='Department')
        count_df[h] = count_df['ZeroCount'] / count_df['Population']
        count_df[h] = count_df[h].round(2)
        count_df = count_df.drop(columns=['ZeroCount', 'Population'])

        if neu_dep_df.empty:
            neu_dep_df = count_df
        else:
            # Merge on 'Department' for subsequent iterations
            neu_dep_df = pd.merge(neu_dep_df, count_df[['Department',h]], on='Department')
    
    neu_dep_df.loc[len(neu_dep_df)] = ['All'] + neu_dep_df[high_perc_Qs].mean().tolist()
    print(neu_dep_df)
    last_row = neu_dep_df.iloc[-1]
    last_row = pd.to_numeric(last_row, errors='coerce')
    top_three = last_row.nlargest(3)
    print(f'Questions with largest percentage of neutral answers (denominator as number of responses of the department):')
    for metric, value in top_three.items():
        print(f'{metric}: {value:.2f}')
    neu_dep_df = neu_dep_df.set_index('Department')

    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(neu_dep_df, annot=True, cmap=sns.cubehelix_palette(as_cmap=True), cbar=True, fmt=".2f")

    # Add titles and labels
    plt.title('Heatmap of Proportions of Neutral Answer by Department')
    plt.xlabel('Questions')

neutral_percentage(combined_dfs,Q_cols)


## Ranking

In [ ]:
# ranking for overall satisfaction, percentage, mean, CV -> visualization
def rank_Qs(df, y):
    # from max to min
    sorted_df = df.sort_values(by = y, ascending=False).reset_index()
    values = sorted_df[y].tolist()
    Q_rank = sorted_df['Q'].tolist()
    # print(values)
    # print(Q_rank)
    print(f'maximum {y} value {values[0]} of is for {sorted_df.loc[0,'Q']}, minimum {y} value {values[-1]} is for {sorted_df.loc[len(values)-1,'Q']}')
    return Q_rank
CV_rank = rank_Qs(Q_CV_df,'CV')
mean_rank = rank_Qs(Q_mean_df,'mean')




# 3. Feature Extraction for Overall Index

## 3.1 Score of overall rating

In [ ]:
# to be checked
list_0 = ["CM", "ISS", "LKYSPP", "Medical School", "UCI"]
list_1 = ["Business School", "CQT", "ETP", "IVO", "ODPRT", "SCALE"]

rating_list = [pd.read_excel(f'Raw Results/NUS {i}.xlsx').iloc[:, -1].dropna()
    for i in list_0] + [pd.read_excel(f'Raw Results/NUS {j}.xlsx').iloc[:, -2].dropna()
    for j in list_1]
# print(rating_list)

rating_list_new = []
# add deparment (to use groupby method)
for i in range(len(rating_list)):
    df = pd.DataFrame(rating_list[i])
    ordered_list = list_0 + list_1
    df['Department'] = ordered_list[i]
    df.columns = ['Overall Rating', 'Department']
    rating_list_new.append(df)
# print(rating_list_new[0])
aggre_rating_df = pd.concat(rating_list_new, ignore_index=True)
print('aggre_rating_df:', aggre_rating_df)


## Box plot
plt.figure(figsize=(10, 6))
sns.boxplot(x='Department', y='Overall Rating', data=aggre_rating_df)
plt.title('Box Plot of Ratings by Department')
plt.xlabel('Department')
plt.ylabel('Overall Rating')
plt.xticks(rotation=45)  # Rotate x labels if needed
# plt.show()
plt.savefig('Graphs/Rating/box_plot.png')


In [ ]:
def visualize_distri_each_dep(y,y_label,path):
    plt.clf()
    x = ["Business School", "CM", "CQT", "ETP", "ISS", "IVO", "LKYSPP", "Medical School", "ODPRT", "SCALE", "UCI"]
    colors = ['blue'] * len(x)
    min_idx = y.index(min(y))
    max_idx = y.index(max(y))
    # Change the color of the bar at index 2 (bar for 'C') to red
    colors[min_idx] = 'red'
    colors[max_idx] = 'green'
    plt.bar(x,y, color = colors, alpha=0.6)
    mean_val = np.mean(y)
    horizontal_line_value = mean_val  # The y-coordinate where the horizontal line will be drawn
    plt.text(x=x[0], y=mean_val, s = round(mean_val,2),  color = 'grey')
    plt.axhline(y=horizontal_line_value, color='grey', linestyle='--', linewidth=2, label='Average score for all departments')
    for i in range(len(x)):
        plt.text(x=x[i], 
                y=y[i], 
                s=f"{y[i]}", 
                ha='center', 
                va='bottom')
    plt.xticks(rotation=45, ha='right') # Rotate x labels for better readability
    y_upper = max(y)+0.01
    y_lower = min(y)-0.01
    # print(f'y range: {y_lower} to {y_upper}')
    plt.ylim(y_lower,y_upper)
    plt.xlabel('Department')
    plt.ylabel(y_label)
    plt.title(f'{y_label} for each department')
    plt.legend()
    plt.show()
    plt.savefig(path)



In [ ]:
# each department 
## Mean of each department
mean_rating_df = aggre_rating_df.groupby('Department')['Overall Rating'].mean().reset_index()
# print('mean_rating_df:', mean_rating_df)
mean_rating_df['Overall Rating'] = mean_rating_df['Overall Rating'].astype('float')
mean_rating_df['Overall Rating'] = round(mean_rating_df['Overall Rating'],2)
overall_rating_score = mean_rating_df['Overall Rating']
# print('overall_rating:', overall_rating)
visualize_distri_each_dep(mean_rating_df['Overall Rating'].tolist(), 'Mean of Overall Rating', 'Graphs/Rating/mean_histogram.png')


## variance
var_rating_df = aggre_rating_df.groupby('Department')['Overall Rating'].var().reset_index()
# print("var_rating_df:", var_rating_df)
var_rating_df['Overall Rating'] = var_rating_df['Overall Rating'].astype('float')
var_rating_df['Overall Rating'] = round(var_rating_df['Overall Rating'],2)
visualize_distri_each_dep(var_rating_df['Overall Rating'].tolist(), 'Variance of Overall Rating', 'Graphs/Rating/var_histogram.png')


## Coefficient of Variation (CV) and Index Score
std_rating_df = aggre_rating_df.groupby('Department')['Overall Rating'].std().reset_index()
std_rating_df.columns = ['Department', 'Standard Deviation']
mean_rating_df.columns = ['Department', 'Mean']
# print("std_rating_df:", std_rating_df)
combined_df = pd.merge(std_rating_df, mean_rating_df, on='Department')
combined_df_CV = combined_df.copy()
combined_df_CV['CV (%)'] = (combined_df_CV['Standard Deviation'] / combined_df_CV['Mean']) * 100


combined_df_score = combined_df.copy()
combined_df_score['Score'] = combined_df_CV['Mean'] - 0.5 * combined_df_CV['Standard Deviation'] 
combined_df_score['Score'] = combined_df_score['Score'].astype('float')
combined_df_score['Score'] = round(combined_df_score['Score'],2)
#  [6.99, 6.74, 6.16, 6.62, 6.79, 7.66, 6.7, 6.88, 6.76, 6.92, 6.95]
visualize_distri_each_dep(combined_df_score['Score'].tolist(), 'Index Score (mean - 0.5*std)', 'Graphs/Rating/index_score_histogram(CV,mean).png')


## conclusion from Score Index: IVO best; Business second best; CQT worst; else average.


## 3.2 Score of 15 questions

In [ ]:
# just use the mean of all Questions
Qs_mean = Q_mean_df['mean'].mean()
Qs_score = round((Qs_mean+2)/4 * 100,2)
print(f'Q_score:{Qs_score}')

# for each department
dep_mean_df = combined_dfs.groupby('Department')[Q_cols].mean().reset_index()
dep_mean_df[Q_cols] = dep_mean_df[Q_cols].astype('float')
dep_mean_df['Qs mean'] = dep_mean_df[Q_cols].mean(axis=1)
dep_mean_df['Qs score'] = (dep_mean_df['Qs mean'] + 2)/4
Qs_score =  round(dep_mean_df['Qs score'],2)

visualize_distri_each_dep(Qs_score.tolist(), 'Score for 15 questions', 'Graphs/Qs/Qs score for each department.png')


## 3.3 Final Score of performance of each department

In [ ]:
# print(mean_rating_df)
mean_Qs_df = dep_mean_df[['Department', 'Qs mean']]
overall_score_df = pd.merge(mean_rating_df,mean_Qs_df, on='Department')
overall_score_df.columns = ['department', 'rating mean', 'Qs mean']
overall_score_df['Qs mean'] = (overall_score_df['Qs mean']+2)/4 * 100
overall_score_df['overall score'] = overall_score_df["rating mean"]*10*0.4 + overall_score_df['Qs mean']*0.6
overall_score_df['overall score'] = round(overall_score_df['overall score'],2)
print(overall_score_df)
overall_score_mean = overall_score_df['overall score'].mean()
print(f'the final score for the whole school: {overall_score_mean.round(2)}')
visualize_distri_each_dep(overall_score_df['overall score'].tolist(), 'Final score', 'Graphs/Fianl score of each department')


# 4. Comparison among departments (hypothesis test, for each Q)

## New (More suitable one) : Kruskal-Wallis Test (for non-normal distribution, rank sum) + Dunn's test (post-hoc)

In [ ]:
# for each Q, check whether there is significant different among departments
signis_qs= []
alpha = 0.05
diff_deps = {key:[] for key in Q_cols}
for q in Q_cols:
    data_list = [combined_dfs[combined_dfs['Department']==d][q] for d in source_list]
    combined_dfs['department_idx'] = combined_dfs['Department'].replace({key:source_list.index(key) for key in source_list})
    result = stats.kruskal(*data_list)
    p = result.pvalue
    # print(f'p value:{p}')
    # p_values.append(p)
    if p < alpha:
        print(f'\n{q} has significant differences. Performing Dunn\'s test...')
        

        # Combine data for Dunn's test
        df = combined_dfs[[q,'department_idx']]

        # Perform Dunn's test
        dunn_results = sp.posthoc_dunn(combined_dfs, val_col=q, group_col='department_idx', p_adjust='bonferroni')
        # print(dunn_results)

        if (dunn_results < alpha).any().any():
            signis_qs.append(q)

            # Initialize a dictionary to count wins for each department
            win_count = {dep: 0 for dep in source_list}
            lose_count = {dep: 0 for dep in source_list}

            # Loop through the Dunn's test results to determine wins
            for i, dep1 in enumerate(dunn_results.index):
                for j, dep2 in enumerate(dunn_results.columns):
                    if i != j:  # Ignore diagonal
                        if dunn_results.loc[dep1, dep2] < alpha:  # Significant difference
                            # Compare medians to decide the winner
                            mean1 = combined_dfs[combined_dfs['Department'] == source_list[i]][q].mean()
                            mean2 = combined_dfs[combined_dfs['Department'] == source_list[j]][q].mean()
                            
                            if mean1 > mean2:  # Assume higher median is better
                                win_count[source_list[dep1]] += 1
                                lose_count[source_list[dep2]] += 1
                            else:
                                win_count[source_list[dep2]] += 1
                                lose_count[source_list[dep1]] += 1

            # Calculate win rate for each department
            total_comparisons = len(source_list) - 1  # Each department is compared to others
            win_rate = {dep: wins / total_comparisons for dep, wins in win_count.items()}
            lose_rate = {dep: loses / total_comparisons for dep, loses in lose_count.items()}

            # Print win rates
            print("Win rates for each department based on Dunn's test results:")
            for department, rate in win_rate.items():
                if rate > 0:
                    print(f"{department}: {rate:.2f}")
            print("Lose rates for each department based on Dunn's test results:")
            for department, rate in lose_rate.items():
                if rate > 0:
                    print(f"{department}: {rate:.2f}")
            
            num_wins = np.unique(list(win_rate.values()))
            if len(num_wins)>1:
                diff_deps[q].append(max(win_rate, key=lambda k: win_rate[k]))
            num_loses = np.unique(list(lose_rate.values()))
            if len(num_loses)>1:
                diff_deps[q].append(max(lose_rate, key=lambda k: lose_rate[k]))
        else:
            print(f'{q}: No significant pairs for Dunns test!')
    else:
        print(f"\n{q} Kruskal-Wallis test is not significant (p = {p}). No further pairwise comparisons needed.")

# Determine which questions have p-values less than alpha
# reject_list = p_values_series[p_values_series < alpha].index.tolist()
# print(f'Qs that are significantly different across departments:{reject_list}')
print(signis_qs)


In [ ]:
# visualization: mean, show which deparments are significantly different
plt.figure(figsize=(15, 10))
ncols = len(signis_qs) // 3 + 1
for i,q in enumerate(signis_qs):
    y = dep_mean_df[q]
    x = source_list
    plt.subplot(3, ncols, i + 1)
    # plt.xlabel('Departments')
    bars = sns.barplot(x=x,y=y)
    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels by 45 degrees
    plt.title(f'Mean of {q} cross departments')
    for j in range(len(y)):
        plt.text(x=x[j], y=y[j],s=f"{y[j]:.2f}", ha='center')
    diff_dep = diff_deps[q]
    diff_dep = [source_list.index(x) for x in diff_dep]
    # print(diff_dep)
    for dep in diff_dep:
        # Access the bar patch using the index and change its color
        if dep in [y.argmax(), y.argmin()]:
            bars.patches[dep].set_facecolor('red')  # Change the color of the specific bar
plt.tight_layout()
plt.show()
plt.savefig('Graphs/Qs/Mean with significant difference cross departments')

# 5. Radar chart for each department

In [ ]:
Q_meaning_dict = {
    'Q1': "Ethical Standards of Management",
    'Q2': "Ethical Standards of Supervisor",
    'Q3': "Sensitivity to Ethical Considerations",
    'Q4': "Trust and Communication",
    'Q5': "Fair Treatment",
    'Q6': "Pressure to Compromise Ethics",
    'Q7': "Internal Controls Effectiveness",
    'Q8': "Disciplinary Actions for Violations",
    'Q9': "Whistleblowing Reporting Knowledge",
    'Q10': "Whistleblowing Channel Confidentiality",
    'Q11': "Protection for Whistleblowers",
    'Q12': "Raising Ethical Concerns",
    'Q13': "Resolution of Internal Complaints",
    'Q14': "Resolution of Public Feedback",
    'Q15': "Ethics Training Programs"
} 

def radar_chart(dep, Q_list, df): # use mean value
    if dep != 'All':
        values = df[df['Department']==dep][Q_list].mean().tolist()
    else:
        values = df[Q_list].mean().tolist()
    # print(values)
    theta = [Q_meaning_dict[q] for q in Q_list]
    rc_df = pd.DataFrame(dict(r=values,
                              theta = theta))
    fig = px.line_polar(rc_df, r='r', theta='theta', line_close=True)
    fig.update_traces(fill='toself')
    fig.update_layout(
    title=f'Performance in {dep} Department',
    title_font_size=20,
    margin=dict(l=60, r=60, t=60, b=60),  # Increase margins to ensure labels fit
    polar=dict(
        angularaxis=dict(
            tickfont_size=12,  # Adjust font size for theta labels
            rotation=90,  # Rotate the chart to better fit labels
            direction="clockwise",  # Optional: make it clockwise
        )
    ),
    width=700,  # Adjust width for compactness
    height=400  # Adjust height for compactness
)
    fig.show()
    # fig.write_image("plotly_chart.png")

# combined_dfs

In [ ]:
radar_chart('Business School', ['Q1','Q5','Q6','Q13','Q14'], combined_dfs)
radar_chart('IVO', ['Q1','Q5','Q6','Q13','Q14'], combined_dfs)
radar_chart('CQT', ['Q1','Q5','Q6','Q13','Q14'], combined_dfs)

In [ ]:
radar_chart('All', ['Q1','Q5','Q6','Q13','Q14'], combined_dfs)

# Wordcloud (for feedbacks)

In [ ]:
feedback_list = [pd.read_excel(f'Raw Results/NUS {source}.xlsx').iloc[:, -1].dropna() for source in source_list]

for i in range(len(feedback_list)):
    fb = pd.DataFrame(feedback_list[i])
    fb = fb.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    # Remove all rows containing the value 'nil'
    fb = fb[~fb.apply(lambda row: row.astype(str).str.contains('nil').any(), axis=1)]
    # print('fb:', fb)
    feedback_list[i] = fb
# print('feedback_list:', feedback_list)


# Function to filter out feedbacks with fewer than 3 words
def filter_feedback(feedback):
    return ' '.join(feedback for feedback in feedback if len(feedback.split()) >= 3)
combined_feedback = ' '.join(
    filter_feedback(feedback_series)
    for feedback_series in feedback_list)


##  Wordcloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(combined_feedback)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # Hide the axis
# plt.show()
wordcloud.to_file('Graphs/Feedbacks/wordcloud.png')

